# Load Dataset #


In [ ]:
!pip install datasets
!pip install evaluate
!pip install accelerate -U

In [1]:
from datasets import load_dataset

imdb = load_dataset("imdb")

/home/nlp_team/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(imdb['train'][0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

# File ultils #

In [21]:
# Load trình Tokenizer cho mô hình
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [22]:
# Định nghĩa hàm xử lý dữ liệu
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [23]:
# Áp dụng hàm xử lý dữ liệu vào bộ dữ liệu
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map: 100%|██████████| 50000/50000 [00:04<00:00, 10024.36 examples/s]


In [24]:
# Áp dụng Padding vào bộ dữ liệu
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
# Định nghĩa cách tính accuracy
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# File Train #


In [26]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, TrainerCallback
from copy import deepcopy
class CustomCallback(TrainerCallback):
    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer

    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy


In [28]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch.utils.data import Subset

class Model_Reactionary_Detection(object):
    def __init__(self, model_name = 'distilbert-base-uncased', train_data_dir = "", val_size = 0.1):
        self.id2label = {0: "NEGATIVE", 1: "POSITIVE"}
        self.label2id = {"NEGATIVE": 0, "POSITIVE": 1}
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name, num_labels=2, id2label=self.id2label, label2id=self.label2id
        )

        self.training_args = TrainingArguments(
            output_dir="./models",
            weight_decay=0.01,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            per_device_train_batch_size=32,
            per_device_eval_batch_size=32,
            learning_rate=0.01,
            num_train_epochs=2
        )

    def train( self, learning_rate = 0.001, EPOCHS = 10, BS = 32,tokenized_imdb=tokenized_imdb):
        trainer = Trainer(
            model=self.model,
            args=self.training_args,
            train_dataset=train_random_samples,
            eval_dataset=test_random_samples,
            tokenizer=tokenizer,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
        )
        trainer.add_callback(CustomCallback(trainer))
        logs = trainer.train()
        return logs



In [29]:
model = Model_Reactionary_Detection()
logs = model.train()
print("Logging after each epoch:")
for log in logs:
    print(log)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.087171,0.528000
2,No log,0.718112,0.472000


Compute metric
Compute metric
Compute metric
Compute metric
Logging after each epoch:
32
2.9979629516601562
{'train_runtime': 827.5593, 'train_samples_per_second': 1.208, 'train_steps_per_second': 0.039, 'total_flos': 132467398656000.0, 'train_loss': 2.9979629516601562, 'epoch': 2.0}


In [ ]:
tokenized_imdb['train'][:1000]

# File Test #

# File Infer #